## Part 4 - Neural Networks and Convolutional Neural Networks

Importing the required libraries and checking for the python version.

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import cv2

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

Loading both the training and testing dataset:

In [2]:
CW_DATASET_PATH = "CW_dataset"

def load_data(dataset_path=CW_DATASET_PATH):
    x_train_all_path = os.path.join(dataset_path, "x_train_all.csv")
    y_train_all_path = os.path.join(dataset_path, "y_train_all.csv")
    x_test_all_path = os.path.join(dataset_path, "x_test_all.csv")
    y_test_all_path = os.path.join(dataset_path, "y_test_all.csv")

    x_train_all = pd.read_csv(x_train_all_path)
    y_train_all = pd.read_csv(y_train_all_path)
    x_test_all = pd.read_csv(x_test_all_path)
    y_test_all = pd.read_csv(y_test_all_path)

    return x_train_all, y_train_all,x_test_all,y_test_all

In [3]:
x_train, y_train,x_test, y_test = load_data()

In [4]:
x_train.shape

(9690, 2304)

In [5]:
y_train.shape

(9690, 1)

In [6]:
x_train = np.asarray(x_train)
x_train = x_train.reshape(x_train.shape[0], -1)
y_train = np.asarray(y_train).ravel()
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

### 1) Linear Classifier

In [7]:
# from sklearn.linear_model import Perceptron

# per_clf = Perceptron(max_iter=1000, tol=1e-3, random_state=42)
# per_clf.fit(x_train, y_train)

# predictions = per_clf.predict(x_test)

In [8]:
# from sklearn.metrics import accuracy_score

# accuracy_p = accuracy_score(y_test, predictions)
# print('Accuracy for Linear Perceptron = ', accuracy_p)

Accuracy for Linear Perceptron =  0.8841423948220065


### 2) Multilayer Perceptron

In [9]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', max_iter=1000, random_state=42)
mlp.fit(x_train, y_train)

mlp_pred = mlp.predict(x_test)

In [10]:
from sklearn.metrics import accuracy_score

accuracy_MLP = accuracy_score(y_test, mlp_pred)
print('Accuracy for Multilayer Perceptron = ', accuracy_MLP)

Accuracy for Multilayer Perceptron =  0.8275080906148867


### 2.1) Experimenting with various Neural Network parameters

In [27]:
# Solver : sgd
# Activation Functions: Logistic, ReLU, tanh
# Number & Size of Layers: (100, 50), (150, 100, 50), (50, 50, 50), (50, 40, 30, 20, 10)
# Learning Rate: constant, invscaling, adaptive
# Epochs: 200, 600, 1000
# Momentum: 0.9, 0.45
pd.set_option('display.max_colwidth', None)

def experiment_parameters_mlp(x_train, y_train, x_test, y_test):
    activ_funcs = ['logistic', 'relu', 'tanh']
    number_layers = [(100,50), (150,100,5), (50,50,50), (50,40,30,20,10)] # (150,100,5), (50,50,50), (50,40,30,20,10)
    learning_rate = ['constant', 'invscaling', 'adaptive'] # 'invscaling', 'adaptive'
    epochs = [200, 600, 1000] # 600, 1000
    momentum_val = [0.49, 0.9] # 0.9

    models = []

    for i in activ_funcs:
        for j in number_layers:
            for k in learning_rate:
                for l in epochs:
                    for m in momentum_val:
                        model = MLPClassifier (
                            #solver='sgd',  
                            activation=i,  
                            hidden_layer_sizes=j,           
                            learning_rate=k,      
                            max_iter=l,    
                            momentum=m,               
                            random_state=42
                        )
                        model.fit(x_train, y_train)
                        models.append(model)

    model_names = []
    accuracies = []
    for n in range(len(models)):
        model_pred = models[n].predict(x_test)
        model_accuracy = accuracy_score(y_test, model_pred)
        accuracies.append(model_accuracy)
        model_names.append(f'Model {n+1}')

    models_names_df = pd.DataFrame({'Model Name': model_names})
    models_df = pd.DataFrame({'Model': models})
    accuracy_df = pd.DataFrame({'Accuracy': accuracies})

    result_df = pd.concat([models_names_df, models_df, accuracy_df], axis=1)
    
    return result_df

In [28]:
result = experiment_parameters_mlp(x_train, y_train, x_test, y_test)
result

,Model Name,Model,Accuracy
0,Model 1,"MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50),\n momentum=0.49, random_state=42)",0.242718
1,Model 2,"MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50),\n random_state=42)",0.242718
2,Model 3,"MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50), max_iter=600,\n momentum=0.49, random_state=42)",0.242718
3,Model 4,"MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50), max_iter=600,\n random_state=42)",0.242718
4,Model 5,"MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50),\n max_iter=1000, momentum=0.49, random_state=42)",0.242718
...,...,...,...
211,Model 212,"MLPClassifier(activation='tanh', hidden_layer_sizes=(50, 40, 30, 20, 10),\n learning_rate='adaptive', random_state=42)",0.242718
212,Model 213,"MLPClassifier(activation='tanh', hidden_layer_sizes=(50, 40, 30, 20, 10),\n learning_rate='adaptive', max_iter=600, momentum=0.49,\n random_state=42)",0.242718
213,Model 214,"MLPClassifier(activation='tanh', hidden_layer_sizes=(50, 40, 30, 20, 10),\n learning_rate='adaptive', max_iter=600, random_state=42)",0.242718
214,Model 215,"MLPClassifier(activation='tanh', hidden_layer_sizes=(50, 40, 30, 20, 10),\n learning_rate='adaptive', max_iter=1000, momentum=0.49,\n random_state=42)",0.242718


In [29]:
pd.set_option('display.max_rows', None)
result

,Model Name,Model,Accuracy
0,Model 1,"MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50),\n momentum=0.49, random_state=42)",0.242718
1,Model 2,"MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50),\n random_state=42)",0.242718
2,Model 3,"MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50), max_iter=600,\n momentum=0.49, random_state=42)",0.242718
3,Model 4,"MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50), max_iter=600,\n random_state=42)",0.242718
4,Model 5,"MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50),\n max_iter=1000, momentum=0.49, random_state=42)",0.242718
5,Model 6,"MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50),\n max_iter=1000, random_state=42)",0.242718
6,Model 7,"MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50),\n learning_rate='invscaling', momentum=0.49, random_state=42)",0.242718
7,Model 8,"MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50),\n learning_rate='invscaling', random_state=42)",0.242718
8,Model 9,"MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50),\n learning_rate='invscaling', max_iter=600, momentum=0.49,\n random_state=42)",0.242718
9,Model 10,"MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50),\n learning_rate='invscaling', max_iter=600, random_state=42)",0.242718


In [64]:
def calculate_train_test_acc(x_train, y_train):
    accuracies = []
    for n in range(len(result)):
        model_pred = result["Model"][n].predict(x_train)
        model_accuracy = accuracy_score(y_train, model_pred)
        accuracies.append(model_accuracy)

    accuracy_df = pd.DataFrame({'Training Accuracy': accuracies})
    test_acc_df = pd.DataFrame({'Testing Accuracy' : result["Accuracy"]})

    result_df = pd.concat([result["Model Name"], result["Model"], accuracy_df, test_acc_df], axis=1)
    
    return result_df

In [65]:
final_df = calculate_train_test_acc(x_train, y_train)
final_df

,Model Name,Model,Training Accuracy,Testing Accuracy
0,Model 1,"MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50),\n momentum=0.49, random_state=42)",0.232198,0.242718
1,Model 2,"MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50),\n random_state=42)",0.232198,0.242718
2,Model 3,"MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50), max_iter=600,\n momentum=0.49, random_state=42)",0.232198,0.242718
3,Model 4,"MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50), max_iter=600,\n random_state=42)",0.232198,0.242718
4,Model 5,"MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50),\n max_iter=1000, momentum=0.49, random_state=42)",0.232198,0.242718
5,Model 6,"MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50),\n max_iter=1000, random_state=42)",0.232198,0.242718
6,Model 7,"MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50),\n learning_rate='invscaling', momentum=0.49, random_state=42)",0.232198,0.242718
7,Model 8,"MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50),\n learning_rate='invscaling', random_state=42)",0.232198,0.242718
8,Model 9,"MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50),\n learning_rate='invscaling', max_iter=600, momentum=0.49,\n random_state=42)",0.232198,0.242718
9,Model 10,"MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50),\n learning_rate='invscaling', max_iter=600, random_state=42)",0.232198,0.242718


In [42]:
#
max_acc = result["Accuracy"].max()
print("Maximum Accuracy = ", max_acc)

max_acc_df = result[result["Accuracy"] == max_acc].copy()

print(f"Total Number of Models with {max_acc} accuracy = ", len(max_acc_df))
max_acc_df

Maximum Accuracy =  0.8773462783171521
Total Number of Models with 0.8773462783171521 accuracy =  18


,Model Name,Model,Accuracy
108,Model 109,"MLPClassifier(hidden_layer_sizes=(50, 50, 50), momentum=0.49, random_state=42)",0.877346
109,Model 110,"MLPClassifier(hidden_layer_sizes=(50, 50, 50), random_state=42)",0.877346
110,Model 111,"MLPClassifier(hidden_layer_sizes=(50, 50, 50), max_iter=600, momentum=0.49,\n random_state=42)",0.877346
111,Model 112,"MLPClassifier(hidden_layer_sizes=(50, 50, 50), max_iter=600, random_state=42)",0.877346
112,Model 113,"MLPClassifier(hidden_layer_sizes=(50, 50, 50), max_iter=1000, momentum=0.49,\n random_state=42)",0.877346
113,Model 114,"MLPClassifier(hidden_layer_sizes=(50, 50, 50), max_iter=1000, random_state=42)",0.877346
114,Model 115,"MLPClassifier(hidden_layer_sizes=(50, 50, 50), learning_rate='invscaling',\n momentum=0.49, random_state=42)",0.877346
115,Model 116,"MLPClassifier(hidden_layer_sizes=(50, 50, 50), learning_rate='invscaling',\n random_state=42)",0.877346
116,Model 117,"MLPClassifier(hidden_layer_sizes=(50, 50, 50), learning_rate='invscaling',\n max_iter=600, momentum=0.49, random_state=42)",0.877346
117,Model 118,"MLPClassifier(hidden_layer_sizes=(50, 50, 50), learning_rate='invscaling',\n max_iter=600, random_state=42)",0.877346


In [54]:
from sklearn.model_selection import cross_val_score

y_test = y_test.ravel()
cross_val = []

for i in range(len(max_acc_df)):
    model = max_acc_df.iloc[i]["Model"]
    scores = cross_val_score(model, x_test, y_test, scoring="accuracy", cv=10)
    cross_val.append(scores.mean())

cross_val_df = pd.DataFrame({"Cross Validation Score" : cross_val})
max_acc_df = max_acc_df.reset_index(drop=True)

cv_df = pd.concat([max_acc_df, cross_val_df], axis=1)
cv_df
    

,Model Name,Model,Accuracy,Cross Validation Score
0,Model 109,"MLPClassifier(hidden_layer_sizes=(50, 50, 50), momentum=0.49, random_state=42)",0.877346,0.847249
1,Model 110,"MLPClassifier(hidden_layer_sizes=(50, 50, 50), random_state=42)",0.877346,0.847249
2,Model 111,"MLPClassifier(hidden_layer_sizes=(50, 50, 50), max_iter=600, momentum=0.49,\n random_state=42)",0.877346,0.847249
3,Model 112,"MLPClassifier(hidden_layer_sizes=(50, 50, 50), max_iter=600, random_state=42)",0.877346,0.847249
4,Model 113,"MLPClassifier(hidden_layer_sizes=(50, 50, 50), max_iter=1000, momentum=0.49,\n random_state=42)",0.877346,0.847249
5,Model 114,"MLPClassifier(hidden_layer_sizes=(50, 50, 50), max_iter=1000, random_state=42)",0.877346,0.847249
6,Model 115,"MLPClassifier(hidden_layer_sizes=(50, 50, 50), learning_rate='invscaling',\n momentum=0.49, random_state=42)",0.877346,0.847249
7,Model 116,"MLPClassifier(hidden_layer_sizes=(50, 50, 50), learning_rate='invscaling',\n random_state=42)",0.877346,0.847249
8,Model 117,"MLPClassifier(hidden_layer_sizes=(50, 50, 50), learning_rate='invscaling',\n max_iter=600, momentum=0.49, random_state=42)",0.877346,0.847249
9,Model 118,"MLPClassifier(hidden_layer_sizes=(50, 50, 50), learning_rate='invscaling',\n max_iter=600, random_state=42)",0.877346,0.847249


In [68]:
from sklearn.model_selection import GridSearchCV

mlp_grid = MLPClassifier()

param_grid = {
    'activation' : ['logistic', 'relu', 'tanh'],
    'hidden_layer_sizes' : [(100,50), (150,100,5), (50,50,50), (50,40,30,20,10)], 
    'learning_rate' : ['constant', 'invscaling', 'adaptive'], 
    'max_iter' : [200, 600, 1000], 
    'momentum' : [0.49, 0.9]
}

gs = GridSearchCV(mlp_grid, param_grid, scoring='accuracy', cv=10, n_jobs=-1)

gs.fit(x_train, y_train)

best_params = gs.best_params_
print("Best parameters: ", best_params)
best_model = gs.best_estimator_
print("Best model: ", best_model)

gs_pred = best_model.predict(x_test)
accuracy_gs = accuracy_score(y_test, gs_pred)
print(f"Accuracy on Test Set: {accuracy_gs:.4f}")

Best parameters:  {'activation': 'relu', 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'invscaling', 'max_iter': 1000, 'momentum': 0.49}
Best model:  MLPClassifier(hidden_layer_sizes=(50, 50, 50), learning_rate='invscaling',
              max_iter=1000, momentum=0.49)
Accuracy on Test Set: 0.8773
